# MTA Data Challenges

In [4]:
from __future__ import division
import csv
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt


### Challenge 1

In [2]:
# !curl -O http://web.mta.info/developers/data/nyct/turnstile/turnstile_150627.txt

In [6]:
with open('turnstile_160109.txt') as f:
    reader = csv.reader(f)
    rows = [[cell.strip() for cell in row] for row in reader]

In [7]:
assert rows.pop(0) == ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME',
                       'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES',
                       'EXITS']

In [8]:
raw_readings = {}
for row in rows:
    raw_readings.setdefault(tuple(row[:4]), []).append(tuple(row[4:]))

In [9]:
raw_readings.items()[0]

(('N134', 'R385', '00-00-00', 'ROCKAWAY BLVD'),
 [('A',
   'IND',
   '01/02/2016',
   '00:00:00',
   'REGULAR',
   '0007142470',
   '0002868386'),
  ('A',
   'IND',
   '01/02/2016',
   '04:00:00',
   'REGULAR',
   '0007142484',
   '0002868393'),
  ('A',
   'IND',
   '01/02/2016',
   '08:00:00',
   'REGULAR',
   '0007142513',
   '0002868412'),
  ('A',
   'IND',
   '01/02/2016',
   '12:00:00',
   'REGULAR',
   '0007142551',
   '0002868438'),
  ('A',
   'IND',
   '01/02/2016',
   '16:00:00',
   'REGULAR',
   '0007142560',
   '0002868453'),
  ('A',
   'IND',
   '01/02/2016',
   '20:00:00',
   'REGULAR',
   '0007142598',
   '0002868480'),
  ('A',
   'IND',
   '01/03/2016',
   '00:00:00',
   'REGULAR',
   '0007142668',
   '0002868490'),
  ('A',
   'IND',
   '01/03/2016',
   '04:00:00',
   'REGULAR',
   '0007142700',
   '0002868493'),
  ('A',
   'IND',
   '01/03/2016',
   '08:00:00',
   'REGULAR',
   '0007142725',
   '0002868499'),
  ('A',
   'IND',
   '01/03/2016',
   '12:00:00',
   'REGULAR

`raw_readings` is a solution to Challenge 1.

### Challenge 2

In [7]:
datetime_cumulative = {turnstile: [(datetime.strptime(date + time,
                                                      '%m/%d/%Y%X'),
                                    int(in_cumulative))
                                   for _, _, date, time,
                                       _, in_cumulative, _ in rows]
                       for turnstile, rows in raw_readings.items()}

In [8]:
for rows in datetime_cumulative.values():
    assert rows == sorted(rows)

AssertionError: 

In [9]:
datetime_count_times = {turnstile: [[rows[i][0],
                                     rows[i+1][1] - rows[i][1],
                                     rows[i+1][0] - rows[i][0]]
                                    for i in range(len(rows) - 1)]
                        for turnstile, rows in datetime_cumulative.items()}

In [10]:
all_counts = [count for rows in datetime_count_times.values() for _, count, _ in rows]
all_counts.sort()
print all_counts[-50:]

[2582, 2583, 2593, 2596, 2613, 2624, 2626, 2654, 2657, 2671, 2677, 2684, 2696, 2697, 2717, 2721, 2734, 2738, 2741, 2766, 2785, 2790, 2830, 2841, 2855, 2860, 2863, 2877, 2909, 2937, 2987, 2995, 3029, 3077, 3116, 3118, 3166, 3198, 3201, 3201, 3390, 85999, 86329, 86599, 1056261, 3144688, 83603316, 117440512, 416445268, 1212629582]


In [ ]:
# print all_counts[:1200]

In [10]:
all_times = [duration.total_seconds() / 60 / 60
             for rows in datetime_count_times.values()
             for _, _, duration in rows]
print Counter(all_times).most_common(10)

[(4.0, 171881), (4.2, 10393), (0.02222222222222222, 254), (8.0, 229), (0.022500000000000003, 227), (4.433333333333334, 160), (2.908333333333333, 56), (0.02277777777777778, 46), (0.023611111111111114, 39), (0.025, 32)]


In [11]:
datetime_counts = {turnstile: [(time, count)
                               for (time, count, _) in rows
                               if 0 <= count <= 5000]
                   for turnstile, rows in datetime_count_times.items()}

`datetime_counts` is a solution to Challenge 2.

In [12]:
all_good_counts = [count for rows in datetime_counts.values() for _, count in rows]
print len(all_good_counts) / len(all_counts)

NameError: name 'all_counts' is not defined

In [13]:
all_good_counts.sort()
print all_good_counts[-5:]

[3166, 3198, 3201, 3201, 3390]


In [14]:
print all_good_counts[:5]

[0, 0, 0, 0, 0]


### Challenge 3

In [15]:
from pprint import pprint
import pandas as pd
day_counts = {}
for turnstile, rows in datetime_counts.items():
    by_day = {}
    for time, count in rows:
        day = time.date()
        by_day[day] = by_day.get(day, 0) + count
    day_counts[turnstile] = sorted(by_day.items())
# pprint (day_counts)

data = pd.DataFrame.from_dict(day_counts,orient='index')



In [16]:
data.head()

,0,1,2,3,4,5,6
"(N134, R385, 00-00-00, ROCKAWAY BLVD)","(2016-01-02, 198)","(2016-01-03, 257)","(2016-01-04, 1240)","(2016-01-05, 1388)","(2016-01-06, 1337)","(2016-01-07, 1396)","(2016-01-08, 1259)"
"(R190, R038, 00-00-02, 215 ST)","(2016-01-02, 312)","(2016-01-03, 269)","(2016-01-04, 630)","(2016-01-05, 609)","(2016-01-06, 660)","(2016-01-07, 664)","(2016-01-08, 611)"
"(PTH08, R540, 00-01-06, PATH WTC)","(2016-01-02, 196)","(2016-01-03, 99)","(2016-01-04, 868)","(2016-01-05, 876)","(2016-01-06, 853)","(2016-01-07, 834)","(2016-01-08, 664)"
"(R159, R164, 01-06-00, 66 ST-LINCOLN)","(2016-01-02, 75)","(2016-01-03, 51)","(2016-01-04, 75)","(2016-01-05, 81)","(2016-01-06, 72)","(2016-01-07, 70)","(2016-01-08, 40)"
"(H016, R250, 00-00-02, GRAND ST)","(2016-01-02, 387)","(2016-01-03, 288)","(2016-01-04, 886)","(2016-01-05, 897)","(2016-01-06, 925)","(2016-01-07, 979)","(2016-01-08, 921)"


In [17]:
nlist = {}
empty = []
for k,v in day_counts.items():
    t = zip(*list(v))
    if len(t) < 2:
        empty.append(tuple([k,v]))
    else:
        nlist[k]=v
        

In [18]:
print len(day_counts),len(nlist)

4571 4544


In [19]:
extreme = []
for k,v in nlist.items():
    t = zip(*list(v))
    dz, cz = t[0], t[1]
    if max(cz) > 5*10**3:
        extreme.append(tuple([max(cz),k,v]))
    

In [20]:
pprint (sorted(extreme,key=lambda count: count[0]))

[(5011,
  ('N063A', 'R011', '00-00-06', '42 ST-PORT AUTH'),
  [(datetime.date(2016, 1, 2), 2219),
   (datetime.date(2016, 1, 3), 1818),
   (datetime.date(2016, 1, 4), 4896),
   (datetime.date(2016, 1, 5), 5011),
   (datetime.date(2016, 1, 6), 4767),
   (datetime.date(2016, 1, 7), 4688),
   (datetime.date(2016, 1, 8), 4427)]),
 (5038,
  ('R221', 'R170', '01-00-02', '14 ST-UNION SQ'),
  [(datetime.date(2016, 1, 2), 2938),
   (datetime.date(2016, 1, 3), 2462),
   (datetime.date(2016, 1, 4), 4589),
   (datetime.date(2016, 1, 5), 4608),
   (datetime.date(2016, 1, 6), 4896),
   (datetime.date(2016, 1, 7), 5038),
   (datetime.date(2016, 1, 8), 4337)]),
 (5054,
  ('N324', 'R018', '00-06-03', 'JKSN HT-ROOSVLT'),
  [(datetime.date(2016, 1, 2), 2299),
   (datetime.date(2016, 1, 3), 1775),
   (datetime.date(2016, 1, 4), 4240),
   (datetime.date(2016, 1, 5), 5054),
   (datetime.date(2016, 1, 6), 3438),
   (datetime.date(2016, 1, 7), 3341),
   (datetime.date(2016, 1, 8), 3254)]),
 (5062,
  ('R417', 

In [ ]:
# 
# plt.show()

In [ ]:
plt.figure(figsize=(10,3))
plt.title(k)
plt.plot(dates,counts)
# for k,v in nlist.items():
#     t = zip(*list(v))
#     dz, cz = t[0], t[1]
#     plt.title(k)
#     plt.plot(dz,cz)
plt.show()

In [ ]:
# plt.figure(figsize=(10,3))
# plt.title(k)
# plt.plot(dates,counts)
for k,v in nlist.items():
    t = zip(*list(v))
    dz, cz = t[0], t[1]
    print dz, cz
    break
#     plt.title(k)
#     plt.plot(dz,cz)
# plt.show()

### Challenge 4

In [ ]:
plt.figure(figsize=(10,3))
extreme = []
empty = []
# print len(day_counts)
for k,v in day_counts.items():
    t = zip(*list(v))
    if len(t)< 2:
        empty.append(tuple([k,v]))
    else:
        dz, cz = t[0], t[1]
        if max(cz) > 10**7:
            extreme.append(tuple([k,v]))
        else:
            plt.title(k)
            plt.plot(dz,cz)

plt.show()

In [ ]:
plt.show()

In [ ]:
data[0][0]

`day_counts` is a solution to Challenge 3.